<a href="https://colab.research.google.com/github/jasmine400/recommender/blob/main/last_releas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

embbeding dependecies

last release


In [ ]:
!pip install -qU sentence-transformers
!pip install -qU wikipedia-api
!pip install -qU hazm
!pip install -qU clean-text[gpl]
!pip install -qU emoji

     |████████████████████████████████| 79 kB 6.7 MB/s 
     |████████████████████████████████| 3.8 MB 47.0 MB/s 
     |████████████████████████████████| 1.2 MB 47.4 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 895 kB 51.7 MB/s 
     |████████████████████████████████| 6.5 MB 46.1 MB/s 
     |████████████████████████████████| 596 kB 48.4 MB/s 
     |████████████████████████████████| 316 kB 22.1 MB/s 
     |████████████████████████████████| 233 kB 53.6 MB/s 
     |████████████████████████████████| 1.4 MB 62.4 MB/s 
     |████████████████████████████████| 175 kB 34.3 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 235 kB 50.9 MB/s 


In [ ]:
!mkdir resources
!wget -q "https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip" -P resources
!unzip -qq resources/resources-0.5.zip -d resources

In [ ]:
!rm -rf /content/4ccae468eb73bf6c4f4de3075ddb5336
!rm -rf /content/preproc
!rm preprocessing.py utils.py
!mkdir -p /content/preproc
!git clone https://gist.github.com/4ccae468eb73bf6c4f4de3075ddb5336.git /content/preproc/
!mv /content/preproc/* /content/
!rm -rf /content/preproc

rm: cannot remove 'preprocessing.py': No such file or directory
rm: cannot remove 'utils.py': No such file or directory
Cloning into '/content/preproc'...
remote: Enumerating objects: 7, done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 7
Unpacking objects: 100% (7/7), done.


In [ ]:
from transformers import BertConfig, BertTokenizer

from preprocessing import cleaning
from IPython import display

import numpy as np
import pandas as pd

import hazm
import requests
import time

import torch
from sentence_transformers import models, SentenceTransformer, util
from sklearn.cluster import KMeans

import pickle

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def rtl_print(outputs, font_size="15px", n_to_br=False):
    outputs = outputs if isinstance(outputs, list) else [outputs]
    if n_to_br:
        outputs = [output.replace('\n', '<br/>') for output in outputs]

    outputs = [f'<p style="text-align: right; direction: rtl; margin-right: 10px; font-size: {font_size};">{output}</p>' for output in outputs]
    display.display(display.HTML(' '.join(outputs)))


def load_st_model(model_name_or_path):
    word_embedding_model = models.Transformer(model_name_or_path)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False)

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model

In [ ]:
# Load the Sentence-Transformer
embedder = load_st_model('m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens')

Downloading:   0%|          | 0.00/519 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/621M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/404 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
#confif bert model
config = BertConfig.from_pretrained(
    'm3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens', **{

    })
print(config.to_json_string())

{
  "_name_or_path": "HooshvareLab/bert-fa-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



In [ ]:
#access google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#access google drive
%cd /content/gdrive/My\ Drive/parsa_clusters
!ls

/content/gdrive/My Drive/parsa_clusters
0.csv	    145.pickle	191.csv     236.pickle	282.csv     57.pickle
0.pickle    146.csv	191.pickle  237.csv	282.pickle  58.csv
100.csv     146.pickle	192.csv     237.pickle	283.csv     58.pickle
100.pickle  147.csv	192.pickle  238.csv	283.pickle  59.csv
101.csv     147.pickle	193.csv     238.pickle	284.csv     59.pickle
101.pickle  148.csv	193.pickle  239.csv	284.pickle  5.csv
102.csv     148.pickle	194.csv     239.pickle	285.csv     5.pickle
102.pickle  149.csv	194.pickle  23.csv	285.pickle  60.csv
103.csv     149.pickle	195.csv     23.pickle	286.csv     60.pickle
103.pickle  14.csv	195.pickle  240.csv	286.pickle  61.csv
104.csv     14.pickle	196.csv     240.pickle	287.csv     61.pickle
104.pickle  150.csv	196.pickle  241.csv	287.pickle  62.csv
105.csv     150.pickle	197.csv     241.pickle	288.csv     62.pickle
105.pickle  151.csv	197.pickle  242.csv	288.pickle  63.csv
106.csv     151.pickle	198.csv     242.pickle	289.csv     63.pickle
106.pickle 

In [ ]:


# pickle_in=open('centers.pickle','rb')
# centroid=pickle.load(pickle_in)

In [ ]:
def corpus_embedding(corpus):
  corpus_embeddings = embedder.encode(corpus, show_progress_bar=True)
  return corpus_embeddings

In [ ]:
#find similar centroids of clusters to full question but for now only one cluster(the most similar one)
#first find Euclidean distance of input to centroids using NumPy then return index of the min distance as a cluster
def find_full_question_clusters(full_question_embedding):
  pickle_in=open('centers.pickle','rb')
  centroid=pickle.load(pickle_in)
  mini=[]
  for i in centroid:
    dist = np.linalg.norm(i - full_question_embedding)
    mini.append(dist)
  cl=min(mini)
  cluster=mini.index(cl)
  return cluster




In [ ]:
def get_title_from_index(index,df):
    return df[df.index == index]["question"].values[0]

In [ ]:
#find most similar question in similar clusters but for now find for only one cluster
def find_most_similar_question_in_cluster(cluster_id,full_question_embedding):
  name=str(cluster_id)
  cluster_df = pd.read_csv('{}.csv'.format(name))

  pickle_in=open('{}.pickle'.format(name),'rb')
  u=pickle.load(pickle_in)
  u = np.append(u, full_question_embedding, axis = 0)
  cosine_sim = cosine_similarity(u)
  question_index=len(cluster_df)
  similar_questions = list(enumerate(cosine_sim[question_index]))
  sorted_similar_questions = sorted(similar_questions, key=lambda x:x[1], reverse=True)
  sorted_similar_questions.pop(0)
  result=[]
  i=0
  for question in sorted_similar_questions:
    q=get_title_from_index(question[0],cluster_df)
    print(q)
    result.append(q)

    i=i+1
    if i>15:
        break
  uniq = list(set(result))
  return uniq

In [ ]:
def recommended_questions(full_question):
  embedding=corpus_embedding(full_question)
  cluster_of_input_question=find_full_question_clusters(embedding)
  recommended_list=find_most_similar_question_in_cluster(cluster_of_input_question,embedding)
  return recommended_list


In [ ]:
corpus=[' دادن خمس سهام اهدایی به فرزند در زمان کودکی ']
a=recommended_questions(corpus)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

پدرم زمینی را به همسر اولش هبه کرد و پس از فوت او این زمین را به نام من سند زد که از همسر دوم ایشان می باشم. آیا این عمل مشکل شرعی دارد؟  
پدرم درباره مصرف یک سوم از اموالش وصیتی داشت و مادرم می‌گوید من وصی پدر هستم، آیا من می‌توانم سهم خود را از ثلث مطالبه کرده و با تشخیص خود صرف امور خیر کنم؟
پدر ما دارای یک قطعه زمین بود که در زمان حیاتش به سه قسمت تقسیم شده بود و با رضایت خودش دو قسمت از آن سه قسمت توسط دو نفر از پسران ایشان ساخته شد که در آن زندگی می کردند.می خواهم بدانم که آیا در تقسیم میراث متوفی ، هر سه قطعه زمین باید محاسبه شود یا فقط همان یک قسمت باقیمانده باید طبق شرع بین دختران و پسران تقسیم شود؟
در تصادفی که پدرم راننده و مقصر بود مادرم فوت کرد. دیه‌ای که بیمه خواهد داد به چه کسانی می‌رسد؟
پدرم چند سال پیش فوت کرد و در وصیت نامه خود مقدار دو میلیون تومان را جهت تهیه جهیزیه برای هر یک از دخترها در نظر گرفت. اکنون بعد از گذشت 13 سال اقدام به تقسیم اموال شده است؟ آیا  این پول به نرخ امروز در نظر گرفته می‌شود؟
پدرم چند سال پیش فوت کرد و در وصیت نامه خود مقدار دو میلیون تومان ر

last_release

In [ ]:
corpus=[' دادن خمس سهام اهدایی به فرزند در زمان کودکی ']
embedd=corpus_embedding(corpus)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
res=find_most_similar_question_in_cluster(209,embedd)

پدرم زمینی را به همسر اولش هبه کرد و پس از فوت او این زمین را به نام من سند زد که از همسر دوم ایشان می باشم. آیا این عمل مشکل شرعی دارد؟  
پدرم درباره مصرف یک سوم از اموالش وصیتی داشت و مادرم می‌گوید من وصی پدر هستم، آیا من می‌توانم سهم خود را از ثلث مطالبه کرده و با تشخیص خود صرف امور خیر کنم؟
پدر ما دارای یک قطعه زمین بود که در زمان حیاتش به سه قسمت تقسیم شده بود و با رضایت خودش دو قسمت از آن سه قسمت توسط دو نفر از پسران ایشان ساخته شد که در آن زندگی می کردند.می خواهم بدانم که آیا در تقسیم میراث متوفی ، هر سه قطعه زمین باید محاسبه شود یا فقط همان یک قسمت باقیمانده باید طبق شرع بین دختران و پسران تقسیم شود؟
در تصادفی که پدرم راننده و مقصر بود مادرم فوت کرد. دیه‌ای که بیمه خواهد داد به چه کسانی می‌رسد؟
پدرم چند سال پیش فوت کرد و در وصیت نامه خود مقدار دو میلیون تومان را جهت تهیه جهیزیه برای هر یک از دخترها در نظر گرفت. اکنون بعد از گذشت 13 سال اقدام به تقسیم اموال شده است؟ آیا  این پول به نرخ امروز در نظر گرفته می‌شود؟
پدرم چند سال پیش فوت کرد و در وصیت نامه خود مقدار دو میلیون تومان ر

In [ ]:
test_list = list(set(res))

In [ ]:
print(len(test_list))

13


In [ ]:
b=recommended_questions(corpus)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(b)

209


In [ ]:
corpus=[' دادن خمس سهام اهدایی به فرزند در زمان کودکی ']
a=recommended_questions(corpus)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(a)

[[ 3.59504908e-01  5.91292799e-01  5.04554868e-01 -1.75822657e-02
   3.48768950e-01  5.97250521e-01 -4.07011598e-01  1.90861568e-01
   3.68746191e-01 -1.62547004e+00  2.27097020e-01 -1.06737234e-01
  -8.28629062e-02 -6.33901879e-02 -2.23623142e-01  7.30039328e-02
  -5.76743782e-01 -3.36264931e-02  3.60207975e-01  1.33556712e+00
   1.67839766e+00 -2.94043452e-01  5.92558444e-01  2.25309983e-01
  -4.84219909e-01 -1.17445636e+00  7.68529892e-01  4.03127640e-01
  -9.21236455e-01 -2.32436013e+00  7.26355493e-01 -1.00551176e+00
   2.87771344e-01 -4.39567268e-01  7.67931342e-01 -6.93742514e-01
   2.11656952e+00  8.59572232e-01 -3.13737313e-03  5.18097520e-01
  -7.90706694e-01  7.95086443e-01  3.07715058e-01 -2.06149042e-01
   2.15781465e-01 -4.71383661e-01 -1.34612426e-01  1.86453438e+00
  -1.11007817e-01  5.71077168e-01 -7.73246944e-01  1.10172951e+00
  -2.09521040e-01 -2.56528646e-01  8.60356390e-01 -2.89907813e-01
   1.05976231e-01 -1.35679990e-01  1.45791745e+00  7.91530550e-01
   2.78991

In [ ]:
mini=[]
for i in centroid:
  dist = np.linalg.norm(i - a)
  mini.append(dist)
cl=min(mini)
cluster=mini.index(cl)
print(cluster)
print(cl)
print(type(cluster))

209
12.871032
<class 'int'>


In [ ]:

# Python code to find Euclidean distance
# using linalg.norm()

import numpy as np
point1 = np.array( [[ 1, 2, 3],
                 [ 4, 2, 5]] )
point2 = np.array( [[ 3, 4, 5],
                 [ 6, 8, 9]] )

# initializing points in
# numpy arrays
# point1 = np.array((1, 2, 3))
# point2 = np.array((1, 1, 1))

# calculating Euclidean distance
# using linalg.norm()
dist = np.linalg.norm(point1 - point2)

# printing Euclidean distance
print(dist)

In [ ]:
def find_similar_question_in_target_clusters():
  pass